#Arrays filteren
##Filtering: het weer in Spanje
We kunnen informatie over de temperatuur en de neerslag per maand downloaden van een aantal Spaanse steden van [Kaggle](https://www.kaggle.com/datasets/alexgczs/monthly-temperature-in-spain-1996-2023).  

In [ ]:
from zipfile import ZipFile
import requests
data = requests.get('https://www.kaggle.com/api/v1/datasets/download/alexgczs/monthly-temperature-in-spain-1996-2023')
with open('data.zip', 'wb') as f:
  f.write(data.content)
with ZipFile('data.zip', 'r') as zip:
  zip.extractall()

We hebben de volgende kolommen: maand, gemiddelde temperatuur, maximum temperatuur, minimum temperatuur, aantal regendagen, hoeveelheid neerslag, gemiddelde wind, plaats

In [ ]:
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
with open(TEMP_CSV, 'r') as f:
  for _ in range(5):
    print(f.readline(), end='')

Om te kunnen filteren op de plaats, moeten we kolom 7 lezen. De functie *np.unique_values()* geeft de unieke plaatsnamen terug.

In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_plaatsen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=7)
np.unique_values(arr_plaatsen)

Stel dat we alleen de gegevens van 'Valencia' willen overhouden. De array *arr_is_valencia* bevat True waar de plaatsnaam 'Valencia' is.

In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_plaatsen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=7)
arr_is_valencia = arr_plaatsen == 'Valencia'


Nu kunnen we de neerslag hoeveelheid lezen (column=5). Er is echter wel een probleem om die data te gebruiken: het woord ' mm'.

Onder het *strings*-attribuut vinden we de NumPy-versies van de Python stringfuncties terug. In dit geval gebruiken we de *strings.replace()*-functie om ' mm' te verwijderen. Met *astype()* zorgen we ervoor dat de strings worden omgezet naar int64.

In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_regen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=5)
arr_regen_valencia = arr_regen[arr_is_valencia]
arr_regen_valencia = np.strings.replace(arr_regen_valencia, 'mm', '').astype(np.int64)
print(f'De gemiddelde regenval in Valencia (mm):{arr_regen_valencia.mean():.2f}' )
#

Om het verschil te zien met het meer regenrijke Galicië kunnen we eens kijken naar Pontevedra.

In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_plaatsen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=7)
arr_is_pontevedra = arr_plaatsen == 'Pontevedra'
arr_regen_pontevedra = arr_regen[arr_is_pontevedra]
arr_regen_pontevedra = np.strings.replace(arr_regen_pontevedra, 'mm', '').astype(np.int64)
print(f'De gemiddelde regenval in Pontevedra (mm): {arr_regen_pontevedra.mean():.2f}')

Achter de schermen worden de vergelijkingsoperatoren vervangen door een UFunc. De ==-operator is hetzelfde als np.equal().

In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_plaatsen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=7)
print(np.equal(arr_plaatsen,'Pontevedra')[:25])

We kunnen een boolean-array gebruiken om te tellen hoe dikwijls een bepaalde waarde voorkomt. Omdat False gelijk is aan 0 kunnen we np.count_nonzero() gebruiken om de True-waarden te tellen. Stel dat we eens naar Lugo willen kijken:

In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_plaatsen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=7)
arr_is_lugo = arr_plaatsen == 'Lugo'
aantal_lugo = np.count_nonzero(arr_is_lugo)
print(f'Aantal keer Lugo voorkomt: {aantal_lugo}')

##Valt er elke maand meer dan 20 mm regen in Lugo?
Eenmaal dat we neerslag hebben voor elke maand in Lugo, kunnen we daarop filteren: > 20.

Om te controleren of alle waarden True zijn, kunnen we np.all() gebruiken. Om te controleren of er minstens 1 waarde True is, gebruiken we np.any().


In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_plaatsen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=7)
arr_regen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=5)
arr_is_lugo = arr_plaatsen == 'Lugo'
arr_regen_lugo = arr_regen[arr_is_lugo]
arr_regen_lugo = np.strings.replace(arr_regen_lugo, 'mm', '').astype(np.int64)
neerslag = 20
print(f'Valt er elke maand meer dan {neerslag} mm in Lugo?', np.all(arr_regen_lugo > neerslag))

## Voorwaarden combineren
Hoeveel maanden viel er meer dan 100 mm regen en was de gemiddelde temperatuur groter dan 5°C? We kunnen de filtering toepassen op de beide arrays (arr_temp_lugo en arr_regen_lugo), maar hoe combineren we die? Met *and*?

Dat werkt dus niet. De and-operator verwacht aan beide kanten een boolean. Maar een array is geen boolean.


In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_plaatsen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=7)
arr_regen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=5)
arr_temp = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=1)
arr_is_lugo = arr_plaatsen == 'Lugo'
arr_regen_lugo = arr_regen[arr_is_lugo]
arr_regen_lugo = np.strings.replace(arr_regen_lugo, 'mm', '').astype(np.int64)
arr_temp_lugo = arr_temp[arr_is_lugo]
arr_temp_lugo = np.strings.replace(arr_temp_lugo, 'ºC', '').astype(np.float64)
neerslag = 100
temp = 5
print(arr_regen_lugo > neerslag)
print(arr_temp_lugo > temp)
print(arr_regen_lugo > neerslag and arr_temp_lugo > temp)


##Bitwise operators
In Python gebruiken we de [bitwise-operators](https://docs.python.org/3/reference/expressions.html#binary-bitwise-operations) om de bits van integers één voor één te vergelijken.

In [ ]:
getal1 = 6               #00000110
getal2 = 5               #00000101
print(getal1 & getal2)   #00000100  AND
print(getal1 | getal2)   #00000111  OR
print(getal1 ^ getal2)   #00000011  XOR
print(~getal1)           #11111001  NOT

In NumPy wordt het concept van 'één-voor-één' toegepast op de elementen van een array. De &-operator (bitwise and) kunnen we gebruiken om een boolean array terug te krijgen met True op de indexen waar beide arrays True(1) zijn.

Let op: de &-operator heeft voorrang op de gewone vergelijkingsoperatoren. We moeten dus ronde haakjes gebruiken.

In [ ]:
import numpy as np
TEMP_CSV = 'monthly_temperature_spain (1996-2023).csv'
arr_plaatsen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=7)
arr_regen = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=5)
arr_temp = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=1)
arr_maanden = np.genfromtxt(TEMP_CSV, delimiter=',', dtype=str, skip_header=1, usecols=0)
arr_is_lugo = arr_plaatsen == 'Lugo'
arr_regen_lugo = arr_regen[arr_is_lugo]
arr_regen_lugo = np.strings.replace(arr_regen_lugo, 'mm', '').astype(np.int64)
arr_temp_lugo = arr_temp[arr_is_lugo]
arr_temp_lugo = np.strings.replace(arr_temp_lugo, 'ºC', '').astype(np.float64)
arr_maanden_lugo = arr_maanden[arr_is_lugo]
neerslag = 100
temp = 5
print(arr_regen_lugo > neerslag)
print(arr_temp_lugo > temp)
print((arr_regen_lugo > neerslag) & (arr_temp_lugo > temp)) #let op de ronde haakjes
print(np.count_nonzero((arr_regen_lugo > neerslag) & (arr_temp_lugo > temp)))
print(arr_maanden_lugo[(arr_regen_lugo > neerslag) & (arr_temp_lugo > temp)])
#